<a href="https://colab.research.google.com/github/enguyen120/BigDataProject/blob/main/Bag_of_Words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/03-TF-IDF-Scikit-Learn.html

is where most of this came from




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Possible place for pre-processing: 

https://subscription.packtpub.com/book/big-data-and-business-intelligence/9781787285217/7/ch07lvl1sec72/identifying-and-removing-rare-words

lemmatization from here: 
https://www.analyticsvidhya.com/blog/2021/06/text-preprocessing-in-nlp-with-python-codes/ 



In [ ]:
# Mount GDrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Do imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
# pd.set_option("max_rows", 600)
from pathlib import Path  

from nltk.stem import WordNetLemmatizer
#defining the object for Lemmatization
wordnet_lemmatizer = WordNetLemmatizer()

import nltk
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# Combine our data into one pandas df, called "df"
# https://www.geeksforgeeks.org/how-to-merge-multiple-csv-files-into-a-single-pandas-dataframe/
filenames = ['/content/drive/MyDrive/Big Data Final/articles1.csv',
              '/content/drive/MyDrive/Big Data Final/articles2.csv',
              '/content/drive/MyDrive/Big Data Final/articles3.csv']
df = pd.concat(map(pd.read_csv, filenames), ignore_index=True)

In [ ]:
# select articles that were published in the november of 2016 
df = df[df.month == 11.0]
df = df[df.year == 2016.0]

In [ ]:
# save df as csv
df.to_csv('/content/drive/MyDrive/Big Data Final/Novdf.csv')

In [ ]:
# Create a TFIDF Vectorizer that takes in strings and gives us a bag of words
# ignores any word with freq < 10
tfidf_vectorizer = TfidfVectorizer(input='content', stop_words='english', min_df = 10)

In [ ]:
# Get a list of strings containing the content of each article ("corpus")
# and a list containing the title of each article ("titles")
corpus = [text for text in df['content']]
titles = [title for title in df['title']]

In [ ]:
# INSERT PREPROCESSING HERE (or after "fit", idk where exactly)
# https://subscription.packtpub.com/book/big-data-and-business-intelligence/9781787285217/7/ch07lvl1sec72/identifying-and-removing-rare-words
stoplist = stopwords.words('english')
corpus2 = []
for article in corpus:
  # tokens = [word.lower() for word in nltk.regexp_tokenize(article, '\w+')]
  tokens = [word.lower() for word in nltk.regexp_tokenize(article, '[a-zA-Z]+')] 
  without_stops = [word for word in tokens if word not in stoplist]
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in without_stops]
  cleaned_text = ' '.join(lemm_text)
  corpus2.append(cleaned_text)

In [ ]:
del corpus
corpus = corpus2

In [ ]:
# Fit the vectorizer to the corpus - ie, the vectorizer now knows what words exist.
tfidf_vectorizer.fit(corpus)

TfidfVectorizer(min_df=10, stop_words='english')

In [ ]:
# # This is what we'll be doing eventually:
max = len(corpus)
numlist = list(range(0, max, 10))

# For now, using 1/10 of the corpus:
# import math
# max = int(math.floor(len(corpus) / 100))
# numlist = list(range(0, max, 10))

#Create the df for the result
tfidf_df = pd.DataFrame()
#For every interval of 10,
for i in range(len(numlist)):
  if(i < len(numlist) - 1):
    # create a bag of words as a sparse matrix
    sparsemat = tfidf_vectorizer.transform(corpus[numlist[i]:numlist[i+1]])
    # get relevant titles
    rownames = titles[numlist[i]:numlist[i+1]]
    # turn BoW into a dense matrix, then into a df
    test_df = pd.DataFrame(sparsemat.toarray(), index=rownames, columns=tfidf_vectorizer.get_feature_names())
    # add it to the df for the result
    tfidf_df = pd.concat([tfidf_df, test_df], axis = 0, ignore_index = True)
    #NOTE: if we want the rows to have the names of the articles, get rid of "ignore_index" above
  else:
    tfidf_vectorizer.transform(corpus[i:max])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# save df of the result
tfidf_df.to_csv('/content/drive/MyDrive/Big Data Final/tfidf_df_pt1.csv')

In [ ]:
# Get top 10 most/least common words
import numpy as np
stupid_dict = {}
for col in tfidf_df.columns:
    stupid_dict[col] = np.sum(tfidf_df[col])
stupid_list = [k for k, v in sorted(stupid_dict.items(), key=lambda item: item[1], reverse = True)]
print("10 most common:", stupid_list[0:10], "\n\n")
print("10 least common:", stupid_list[-10:])

10 most common: ['trump', 'said', 'clinton', 'state', 'people', 'election', 'year', 'president', 'new', 'say'] 


10 least common: ['tolerable', 'compulsion', 'zeroed', 'herring', 'pettiness', 'vindictiveness', 'subsumed', 'unproductive', 'preface', 'perpetuates']


code scraps below ⬇️ --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# tfidf_vectorizer.get_feature_names()

In [ ]:
# print df for the result
print(tfidf_df)

      aaron  aaronkleinshow   ab  aback  abadi  abandon  abandoned  \
0       0.0             0.0  0.0    0.0    0.0      0.0   0.015958   
1       0.0             0.0  0.0    0.0    0.0      0.0   0.000000   
2       0.0             0.0  0.0    0.0    0.0      0.0   0.000000   
3       0.0             0.0  0.0    0.0    0.0      0.0   0.019500   
4       0.0             0.0  0.0    0.0    0.0      0.0   0.000000   
...     ...             ...  ...    ...    ...      ...        ...   
7975    0.0             0.0  0.0    0.0    0.0      0.0   0.000000   
7976    0.0             0.0  0.0    0.0    0.0      0.0   0.000000   
7977    0.0             0.0  0.0    0.0    0.0      0.0   0.000000   
7978    0.0             0.0  0.0    0.0    0.0      0.0   0.056128   
7979    0.0             0.0  0.0    0.0    0.0      0.0   0.000000   

      abandoning  abandonment  abbas  ...  zimbabwe  zionist  zip  zombie  \
0            0.0          0.0    0.0  ...       0.0      0.0  0.0     0.0   
1    

In [ ]:
# stoplist

In [ ]:
# print(df)

In [ ]:
# import pandas as pd
# test_df = pd.DataFrame()
# test_df_2 = pd.DataFrame(data={'col1': [1, 2], 'col2': [3, 4]})
# test_df_3 = pd.concat([test_df, test_df_2], axis = 0, ignore_index = True)
# print(test_df_3)
# test_df_4 = pd.DataFrame(data={'col1': [8, 9], 'col2': [13, 0]})
# test_df_3 = pd.concat([test_df_3, test_df_4], axis = 0, ignore_index = True)
# print(test_df_3)

In [ ]:
# numlist = list(range(0, 132, 10))
# for i in range(len(numlist)):
#   if(i < len(numlist) - 1):
#     print(numlist[i], numlist[i+1])
#   else:
#     print(numlist[i], 132)

In [ ]:
# densearray = tfidf_vector.toarray() #this is where the crash happens

In [ ]:
# tfidf_df = pd.DataFrame(tfidf_vector.toarray(), index=titles, columns=tfidf_vectorizer.get_feature_names())


#TESTS TO MAKE IT SMALLER
# for i in range(0, tfidf_vector.shape[0], 10):
#   tfidf_df = pd.DataFrame(tfidf_vector[i - 10, i].toarray(), index=titles, columns=tfidf_vectorizer.get_feature_names())

# while(i > 0):
#   if(i > 10):
#     tfidf_df = pd.DataFrame(tfidf_vector[i - 10, i].toarray(), index=titles, columns=tfidf_vectorizer.get_feature_names())
#     i -= 10
#   else:
#     tfidf_df = pd.DataFrame(tfidf_vector[0, i].toarray(), index=titles, columns=tfidf_vectorizer.get_feature_names())
#     i = 0